## Imports

In [1]:
import numpy as np
import pickle as pkl
import pandas as pd

import os

%matplotlib inline

In [2]:
pkl_dir = "pkl/"
pkl_files = os.listdir(pkl_dir)

## Run 05

In [72]:
r5_files = [f for f in pkl_files if "run_05" in f]

In [73]:
#r5_files

In [74]:
df = []

In [75]:
for f in r5_files:
    df_ = pd.read_pickle(pkl_dir+f)
    df.append(df_)

In [76]:
df = pd.concat(df, axis=0, ignore_index=True)

In [77]:
#df

Interpolate B field!

In [78]:
from mu2e.tools.mapinterp import get_df_interp_func

In [79]:
from mu2e import mu2e_ext_path
from mu2e.dataframeprod import DataFrameMaker

In [80]:
df_B = DataFrameMaker(mu2e_ext_path+"datafiles/Mau13/Mu2e_DSMap_V13",input_type='pkl').data_frame

In [81]:
df_B.eval("Bx = Bx / 10000",inplace=True)
df_B.eval("By = By / 10000",inplace=True)
df_B.eval("Bz = Bz / 10000",inplace=True)

In [82]:
df_B.eval("X = X * 1000",inplace=True)
df_B.eval("Y = Y * 1000",inplace=True)
df_B.eval("Z = Z * 1000",inplace=True)

In [83]:
bfield_interp_mm = get_df_interp_func(df_B)

In [84]:
#bfield_interp_mm(40, 11, 5922, True)

In [85]:
df["B_tuple"] = [bfield_interp_mm(row.X,row.Y,row.Z) for row in df.itertuples()]

In [86]:
df["Bx"] = [row.B_tuple[0] for row in df.itertuples()]
df["By"] = [row.B_tuple[1] for row in df.itertuples()]
df["Bz"] = [row.B_tuple[2] for row in df.itertuples()]

In [87]:
from mu2e.tools.particletransport import *

In [88]:
#state = list(test[["X","Y","Z","Vx","Vy","Vz"]].values)

In [89]:
#f = lorentz_force(state, 0, bfield_interp_mm)[3:]

In [90]:
#f

In [91]:
#state[:3]

In [92]:
df["Force_tuple"] = [lorentz_force(list(row)[1:], 0, bfield_interp_mm)[3:] for row in df[["X","Y","Z","Vx","Vy","Vz"]].itertuples()]

In [93]:
df["F"] = [(row.Force_tuple[0]**2 + row.Force_tuple[1]**2 + row.Force_tuple[2]**2)**(1/2) for row in df.itertuples()]

In [94]:
#df.F.describe()

In [95]:
#df.F.hist()

In [96]:
df.eval("V = (Vx**2 + Vy**2 + Vz**2)**(1/2)", inplace=True)
df.eval("B = (Bx**2 + By**2 + Bz**2)**(1/2)", inplace=True)

In [97]:
#df.V.hist()

In [98]:
#df.V.describe()

In [99]:
df["Fx"] = [row.Force_tuple[0] for row in df.itertuples()]
df["Fy"] = [row.Force_tuple[1] for row in df.itertuples()]
df["Fz"] = [row.Force_tuple[2] for row in df.itertuples()]

In [100]:
df.columns

Index(['X', 'Y', 'Z', 'Vx', 'Vy', 'Vz', 'time', 'event', 'E0', 'EF', 'B_tuple',
       'Bx', 'By', 'Bz', 'Force_tuple', 'F', 'V', 'B', 'Fx', 'Fy', 'Fz'],
      dtype='object')

In [106]:
df.event.unique()

array([ 6, 15, 12,  2])

In [102]:
df.max()

X                                                     552.42
Y                                                    524.042
Z                                                    12001.8
Vx                                               2.99306e+11
Vy                                               2.99769e+11
Vz                                               2.90545e+11
time                                                   8e-08
event                                                     15
E0                                                   105.001
EF                                                   105.001
B_tuple    (0.10742643625551421, 0.08219499994081358, 2.1...
Bx                                                  0.107426
By                                                 0.0976276
Bz                                                   2.13079
F                                                5.47708e+20
V                                                2.99789e+11
B                       

In [103]:
df["Fx_scaled"] = df["Fx"] / df.F.max()
df["Fx_scaled"] = df["Fx"] / df.F.max()
df["Fx_scaled"] = df["Fx"] / df.F.max()

df["Fx_scaled"] = df["Fx"] / df.F.max()
df["Fx_scaled"] = df["Fx"] / df.F.max()
df["Fx_scaled"] = df["Fx"] / df.F.max()

In [104]:
for i in ["Fx","Fy","Fz","Vx","Vy","Vz","Bx","By","Bz"]:
    df[f"{i}_scaled"] = df[i] / df[i[0]].max()

## Write to JSON File

In [105]:
df.to_json("track_data.json", orient="records")